In [99]:
import nltk
import pandas as pd
import numpy as np
from nltk import sent_tokenize, word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [100]:
wpt = WordPunctTokenizer()
tbwt = TreebankWordTokenizer()
p_stem = PorterStemmer()
w_lem = WordNetLemmatizer()

In [39]:
df = pd.read_csv('train.csv')

In [6]:
len(df)

159571

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [8]:
df.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [118]:
df_toxic = df[df.toxic == 1]
df_severe_toxic = df[df.severe_toxic == 1]
df_obscene = df[df.obscene == 1]
df_threat = df[df.threat == 1]
df_insult = df[df.insult == 1]
df_identity_hate = df[df.identity_hate == 1]
meaningless_words = ['wikipedia','i','hi','like','u','me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']
meaningless_punc = ['''"''','''""''',"""'""",'/',':','-','(',')','?','.',',']

In [119]:
# 1-word / single-word count

def word_count_1(dataframe,name):

    global_list_cnt = []
    
    for num in range(len(dataframe.comment_text)):

        place_holder = []
        local_list_cnt = []
        
        for word in wpt.tokenize(dataframe.comment_text.iloc[num]):
        #for word in tbwt.tokenize(dataframe.comment_text.iloc[num]):
            
            word_low = word.lower()
            
            # Taking out the meaningless words (only for 1-word count)
            if word_low not in meaningless_words and word_low not in meaningless_punc:
                
                #place_holder.append(p_stem.stem(word_low))
                place_holder.append(word_low)

                if len(place_holder) == 1:
                    phrase = " ".join(place_holder)  

                    if phrase not in local_list_cnt:
                        local_list_cnt.append(phrase)

                    del place_holder[0]     
        
        global_list_cnt.extend(local_list_cnt)
    
    list_cnt = Counter(global_list_cnt)
        
    df_wcnt = pd.DataFrame.from_dict(dict(list_cnt),orient='index',columns=['count'])
    #df_wcnt['% Total'] = round(df_wcnt.Count / df_wcnt.Count.sum() * 100,1)

    rank = []
    for num in range(50):
        rank.append(num+1)    
   
    df_wcnt = df_wcnt.sort_values('count',ascending=False).head(50).reset_index()
    df_wcnt['rank']=rank
    df_wcnt_ranked = df_wcnt.set_index('rank')
    df_wcnt_ranked = df_wcnt_ranked.rename(columns={"index": name})
  
    return (df_wcnt_ranked)  

In [101]:
# 2-word / word-pair count

def word_count_2(dataframe,name):

    global_list_cnt = []
    
    for num in range(len(dataframe.comment_text)):

        place_holder = []
        local_list_cnt = []
        
        #for word in wpt.tokenize(dataframe.comment_text.iloc[num]):
        for word in tbwt.tokenize(dataframe.comment_text.iloc[num]):

            word_low = word.lower()
            place_holder.append(word_low)
    
            if len(place_holder) == 2:
                phrase = " ".join(place_holder)  
                
                if phrase not in local_list_cnt:
                    local_list_cnt.append(phrase)
              
                del place_holder[0]     
        
        global_list_cnt.extend(local_list_cnt)
    
    list_cnt = Counter(global_list_cnt)
        
    df_wcnt = pd.DataFrame.from_dict(dict(list_cnt),orient='index',columns=['count'])
    #df_wcnt['% Total'] = round(df_wcnt.Count / df_wcnt.Count.sum() * 100,1)

    rank = []
    for num in range(50):
        rank.append(num+1)    
   
    df_wcnt = df_wcnt.sort_values('count',ascending=False).head(50).reset_index()
    df_wcnt['rank']=rank
    df_wcnt_ranked = df_wcnt.set_index('rank')
    df_wcnt_ranked = df_wcnt_ranked.rename(columns={"index": name})
  
    return (df_wcnt_ranked)     

In [102]:
# 3-word count

def word_count_3(dataframe,name):

    global_list_cnt = []
    
    for num in range(len(dataframe.comment_text)):

        place_holder = []
        local_list_cnt = []
        
        #for word in wpt.tokenize(dataframe.comment_text.iloc[num]):
        for word in tbwt.tokenize(dataframe.comment_text.iloc[num]):
            
            word_low = word.lower()
            place_holder.append(word_low)
    
            if len(place_holder) == 3:
                phrase = " ".join(place_holder)  
                
                if phrase not in local_list_cnt:
                    local_list_cnt.append(phrase)
              
                del place_holder[0]     
        
        global_list_cnt.extend(local_list_cnt)
    
    list_cnt = Counter(global_list_cnt)
        
    df_wcnt = pd.DataFrame.from_dict(dict(list_cnt),orient='index',columns=['count'])
    #df_wcnt['% Total'] = round(df_wcnt.Count / df_wcnt.Count.sum() * 100,1)

    rank = []
    for num in range(50):
        rank.append(num+1)    
   
    df_wcnt = df_wcnt.sort_values('count',ascending=False).head(50).reset_index()
    df_wcnt['rank']=rank
    df_wcnt_ranked = df_wcnt.set_index('rank')
    df_wcnt_ranked = df_wcnt_ranked.rename(columns={"index": name})
  
    return (df_wcnt_ranked)  

In [103]:
# 4-word count

def word_count_4(dataframe,name):

    global_list_cnt = []
    
    for num in range(len(dataframe.comment_text)):

        place_holder = []
        local_list_cnt = []
        
        #for word in wpt.tokenize(dataframe.comment_text.iloc[num]):
        for word in tbwt.tokenize(dataframe.comment_text.iloc[num]):
            
            word_low = word.lower()
            place_holder.append(word_low)
    
            if len(place_holder) == 4:
                phrase = " ".join(place_holder)  
                
                if phrase not in local_list_cnt:
                    local_list_cnt.append(phrase)
              
                del place_holder[0]     
        
        global_list_cnt.extend(local_list_cnt)
    
    list_cnt = Counter(global_list_cnt)
        
    df_wcnt = pd.DataFrame.from_dict(dict(list_cnt),orient='index',columns=['count'])
    #df_wcnt['% Total'] = round(df_wcnt.Count / df_wcnt.Count.sum() * 100,1)

    rank = []
    for num in range(50):
        rank.append(num+1)    
   
    df_wcnt = df_wcnt.sort_values('count',ascending=False).head(50).reset_index()
    df_wcnt['rank']=rank
    df_wcnt_ranked = df_wcnt.set_index('rank')
    df_wcnt_ranked = df_wcnt_ranked.rename(columns={"index": name})
  
    return (df_wcnt_ranked)

In [120]:
df_toxic_1wcnt = word_count_1(df_toxic,'toxic')
df_severe_toxic_1wcnt = word_count_1(df_severe_toxic,'severe_toxic')
df_obscene_1wcnt = word_count_1(df_obscene,'obscene')
df_threat_1wcnt = word_count_1(df_threat,'threat')
df_insult_1wcnt = word_count_1(df_insult,'insult')
df_identity_hate_1wcnt = word_count_1(df_identity_hate,'identity_hate')

In [104]:
df_toxic_2wcnt = word_count_2(df_toxic,'toxic')
df_severe_toxic_2wcnt = word_count_2(df_severe_toxic,'severe_toxic')
df_obscene_2wcnt = word_count_2(df_obscene,'obscene')
df_threat_2wcnt = word_count_2(df_threat,'threat')
df_insult_2wcnt = word_count_2(df_insult,'insult')
df_identity_hate_2wcnt = word_count_2(df_identity_hate,'identity_hate')    

In [105]:
df_toxic_3wcnt = word_count_3(df_toxic,'toxic')
df_severe_toxic_3wcnt = word_count_3(df_severe_toxic,'severe_toxic')
df_obscene_3wcnt = word_count_3(df_obscene,'obscene')
df_threat_3wcnt = word_count_3(df_threat,'threat')
df_insult_3wcnt = word_count_3(df_insult,'insult')
df_identity_hate_3wcnt = word_count_3(df_identity_hate,'identity_hate')

In [108]:
df_toxic_4wcnt = word_count_4(df_toxic,'toxic')
df_severe_toxic_4wcnt = word_count_4(df_severe_toxic,'severe_toxic')
df_obscene_4wcnt = word_count_4(df_obscene,'obscene')
df_threat_4wcnt = word_count_4(df_threat,'threat')
df_insult_4wcnt = word_count_4(df_insult,'insult')
df_identity_hate_4wcnt = word_count_4(df_identity_hate,'identity_hate')

In [121]:
master_list_1wc = pd.concat([df_toxic_1wcnt, df_severe_toxic_1wcnt,df_obscene_1wcnt,df_threat_1wcnt,df_insult_1wcnt,df_identity_hate_1wcnt],axis=1)
master_list_2wc = pd.concat([df_toxic_2wcnt, df_severe_toxic_2wcnt,df_obscene_2wcnt,df_threat_2wcnt,df_insult_2wcnt,df_identity_hate_2wcnt],axis=1)
master_list_3wc = pd.concat([df_toxic_3wcnt, df_severe_toxic_3wcnt,df_obscene_3wcnt,df_threat_3wcnt,df_insult_3wcnt,df_identity_hate_3wcnt],axis=1)
master_list_4wc = pd.concat([df_toxic_4wcnt, df_severe_toxic_4wcnt,df_obscene_4wcnt,df_threat_4wcnt,df_insult_4wcnt,df_identity_hate_4wcnt],axis=1)

In [122]:
# 1-word / single-word count ; took out meaningless words ; no stemming
master_list_1wc

,toxic,count,severe_toxic,count,obscene,count,threat,count,insult,count,identity_hate,count
rank,,,,,,,,,,,,
1,!,2868,fuck,707,fuck,2394,kill,118,fuck,1863,fuck,307
2,fuck,2463,fucking,418,!,1561,die,115,!,1506,gay,265
3,get,1680,!,311,fucking,1541,!,112,fucking,1216,!,255
4,fucking,1621,shit,256,shit,1097,fuck,88,go,983,fucking,228
5,go,1536,go,249,go,999,fucking,87,get,890,go,167
6,page,1522,bitch,228,get,946,going,66,shit,826,faggot,162
7,know,1338,ass,190,ass,807,go,65,page,679,shit,160
8,people,1335,get,171,page,776,hope,60,ass,656,get,150
9,shit,1276,suck,168,bitch,701,shit,58,bitch,645,nigger,135


In [111]:
# 2-word / word-pair count
master_list_2wc

,toxic,count,severe_toxic,count,obscene,count,threat,count,insult,count,identity_hate,count
rank,,,,,,,,,,,,
1,you are,2126,fuck you,292,you are,997,i will,114,you are,1240,you are,208
2,do n't,1778,! !,223,", you",896,you are,68,", you",969,! !,161
3,'' '',1728,you are,173,do n't,896,going to,63,do n't,809,is a,156
4,", you",1692,", you",167,'' '',832,! !,62,! !,756,", you",139
5,`` '',1654,do n't,109,! !,812,", i",61,you 're,681,are a,112
6,", i",1479,a fucking,109,`` '',790,kill you,58,", i",667,", and",111
7,", and",1455,", i",104,", i",762,if you,51,'' '',665,'' '',110
8,! !,1373,you 're,102,fuck you,723,i 'm,50,fuck you,640,fuck you,107
9,is a,1206,you fucking,100,", and",691,", you",50,", and",636,`` '',103


In [112]:
# 3-word count
master_list_3wc

,toxic,count,severe_toxic,count,obscene,count,threat,count,insult,count,identity_hate,count
rank,,,,,,,,,,,,
1,! ! !,1074,! ! !,187,! ! !,658,! ! !,52,! ! !,613,! ! !,134
2,you are a,677,you are a,86,you are a,375,i hope you,35,you are a,487,you are a,92
3,i do n't,536,go fuck yourself,70,i do n't,297,am going to,25,you 're a,255,you 're a,35
4,you do n't,373,piece of shit,51,you 're a,224,you are a,25,i do n't,235,you do n't,27
5,you 're a,344,you 're a,48,go fuck yourself,178,i am going,22,you do n't,173,go fuck yourself,27
6,'' '' .,285,son of a,39,you do n't,177,i will kill,20,go fuck yourself,163,piece of shit,24
7,", you are",261,fuck you !,35,piece of shit,152,going to kill,19,get a life,143,", you are",21
8,do n't you,251,"fuck you ,",34,do n't you,139,hope you die,18,", you are",141,i do n't,19
9,? ? ?,248,i hope you,32,what the fuck,139,will kill you,17,piece of shit,139,get a life,19


In [113]:
# 4-word count
master_list_4wc

,toxic,count,severe_toxic,count,obscene,count,threat,count,insult,count,identity_hate,count
rank,,,,,,,,,,,,
1,! ! ! !,593,! ! ! !,110,! ! ! !,360,! ! ! !,27,! ! ! !,338,! ! ! !,73
2,why do n't you,158,son of a bitch,30,why do n't you,97,i am going to,22,why do n't you,98,why do n't you,13
3,? ? ? ?,111,shut the fuck up,23,son of a bitch,76,i hope you die,18,son of a bitch,74,! ! ! i,12
4,do n't give a,101,you ! ! !,20,do n't give a,75,i will kill you,16,", you are a",57,! ! ! you,11
5,", i do n't",99,! ! ! you,19,shut the fuck up,60,i 'm going to,15,! ! ! you,52,son of a bitch,11
6,i 'm going to,95,you son of a,18,i do n't give,58,you ! ! !,13,! ! ! i,52,", you are a",11
7,! ! ! i,89,! ! ! i,18,i 'm going to,54,going to kill you,12,i 'm going to,52,i am going to,10
8,i am going to,86,why do n't you,18,! ! ! i,52,am going to kill,9,shut the fuck up,50,because you are a,9
9,if you do n't,85,you are a fucking,17,", i do n't",51,if you do n't,9,you 're a fucking,46,? ? ? ?,8
